In [6]:
from selenium import webdriverfrom selenium.webdriver.common.by import Byfrom selenium.webdriver.chrome.service import Servicefrom selenium.webdriver.support.ui import WebDriverWaitfrom selenium.webdriver.support import expected_conditions as ECimport osimport timeimport glob# 📌 CONFIGURAÇÃOINICIAR_APARTIR_DO_ITEM = 0  # Altere este valor para pular os primeiros N downloads# Caminho da pasta onde os arquivos devem ser salvosimport sysfrom pathlib import Pathsys.path.append(str(Path().absolute().parent.parent / "src"))from utils.config import DFP_2024_PATHpasta_destino = str(DFP_2024_PATH)os.makedirs(pasta_destino, exist_ok=True)# Função para checar se o arquivo já existedef arquivo_ja_existe(nome_base, pasta):    padrao = os.path.join(pasta, f"{os.path.splitext(nome_base)[0]}*")    arquivos_existentes = glob.glob(padrao)    return any(        arq.lower().endswith(".zip") and os.path.basename(arq).startswith(os.path.splitext(nome_base)[0])        for arq in arquivos_existentes    )# Espera até que todos os arquivos .crdownload desapareçamdef esperar_fim_downloads(download_path, timeout=120):    inicio = time.time()    while True:        downloads_em_andamento = glob.glob(os.path.join(download_path, "*.crdownload"))        if not downloads_em_andamento:            return True        if time.time() - inicio > timeout:            print("⏰ Timeout esperando downloads finalizarem.")            return False        time.sleep(1)# Configuração do driverservice = Service()options = webdriver.ChromeOptions()prefs = {"download.default_directory": pasta_destino}options.add_experimental_option("prefs", prefs)driver = webdriver.Chrome(service=service, options=options)wait = WebDriverWait(driver, 30)try:    driver.get("https://www.rad.cvm.gov.br/ENET/frmConsultaExternaCVM.aspx")    time.sleep(5)    wait.until(EC.element_to_be_clickable((By.ID, "rdPeriodo"))).click()    wait.until(EC.presence_of_element_located((By.ID, "txtDataIni"))).send_keys("01/01/2024")    wait.until(EC.presence_of_element_located((By.ID, "txtDataFim"))).send_keys("01/08/2025")    wait.until(EC.presence_of_element_located((By.ID, "txtDataReferencia"))).send_keys("30/11/2024")    driver.find_element(By.ID, "cboCategorias_chosen").click()    time.sleep(1)    wait.until(EC.presence_of_element_located((By.ID, "cboCategorias_chosen_input"))).send_keys("DFP")    time.sleep(2)    for opcao in driver.find_elements(By.CSS_SELECTOR, "#cboCategorias_chosen .chosen-drop ul li.active-result"):        if "DFP - Demonstrações Financeiras Padronizadas" in opcao.text:            opcao.click()            break    wait.until(EC.element_to_be_clickable((By.ID, "btnConsulta"))).click()    time.sleep(10)    def ordenar_por_nome_empresa():        try:            th_empresa = wait.until(EC.element_to_be_clickable((By.XPATH, "//th[contains(text(), 'Empresa')]")))            ordem_atual = th_empresa.get_attribute("aria-sort")            if ordem_atual != "ascending":                driver.execute_script("arguments[0].click();", th_empresa)                time.sleep(3)        except Exception as e:            print(f"⚠️ Falha ao ordenar empresas: {e}")    def baixar_todos_downloads():        pagina = 1        contador_downloads = 1        ordenar_por_nome_empresa()        while True:            print(f"\n📄 Página {pagina}")            tabela = wait.until(EC.presence_of_element_located((By.ID, "grdDocumentos")))            linhas = tabela.find_elements(By.CSS_SELECTOR, "tbody tr")            headers = tabela.find_elements(By.CSS_SELECTOR, "thead th")            indice_status = next((i for i, th in enumerate(headers) if "Status" in th.text), None)            if indice_status is None:                print("❌ Coluna 'Status' não encontrada.")                break            for linha in linhas:                colunas = linha.find_elements(By.TAG_NAME, "td")                if len(colunas) <= indice_status:                    continue                status = colunas[indice_status].text.strip()                if status != "Ativo":                    continue                if contador_downloads < INICIAR_APARTIR_DO_ITEM:                    contador_downloads += 1                    continue                try:                    botao = linha.find_element(By.CSS_SELECTOR, "i.fi-download")                    onclick = botao.get_attribute("onclick")                    doc_name = onclick.split(",")[2].replace("'", "").strip()                    if arquivo_ja_existe(doc_name, pasta_destino):                        print(f"🟡 {doc_name} já existe. Pulando.")                        continue                    print(f"{contador_downloads:03d} ⬇️ Iniciando download: {doc_name}")                    driver.execute_script("arguments[0].click();", botao)                    if esperar_fim_downloads(pasta_destino):                        print(f"✅ Download finalizado: {doc_name}")                    else:                        print(f"⛔ Timeout: {doc_name} pode não ter finalizado corretamente.")                    contador_downloads += 1                except Exception as e:                    print(f"❌ Erro ao tentar baixar: {e}")            try:                botao_proxima = driver.find_element(By.CSS_SELECTOR, "a.paginate_button.next:not(.disabled)")                driver.execute_script("arguments[0].click();", botao_proxima)                pagina += 1                time.sleep(8)            except:                print("✅ Fim da paginação.")                break    baixar_todos_downloads()finally:    print("🧹 Finalizando driver...")    driver.quit()    # 🔎 Remover arquivos duplicados com (1).zip no nome    for arquivo in os.listdir(pasta_destino):        if arquivo.endswith("(1).zip"):            caminho = os.path.join(pasta_destino, arquivo)            try:                os.remove(caminho)                print(f"🗑️ Removido duplicado: {arquivo}")            except Exception as e:                print(f"⚠️ Erro ao remover {arquivo}: {e}")


📄 Página 1
001 ⬇️ Iniciando download: 024058DFP311220240200146392-66
✅ Download finalizado: 024058DFP311220240200146392-66
002 ⬇️ Iniciando download: 096393DFP311220240100146507-73
✅ Download finalizado: 096393DFP311220240100146507-73
003 ⬇️ Iniciando download: 018376DFP311220240100144965-75
✅ Download finalizado: 018376DFP311220240100144965-75
004 ⬇️ Iniciando download: 024910DFP311220240100145376-78
✅ Download finalizado: 024910DFP311220240100145376-78
005 ⬇️ Iniciando download: 025224DFP311220240200149563-69
✅ Download finalizado: 025224DFP311220240200149563-69
006 ⬇️ Iniciando download: 016284DFP311220240100145737-61
✅ Download finalizado: 016284DFP311220240100145737-61
007 ⬇️ Iniciando download: 023396DFP311220240200145588-61
✅ Download finalizado: 023396DFP311220240200145588-61
008 ⬇️ Iniciando download: 025283DFP311220240300145835-65
✅ Download finalizado: 025283DFP311220240300145835-65
009 ⬇️ Iniciando download: 022179DFP311220240100144832-61
✅ Download finalizado: 022179DFP31